In [1]:
function nr(r) 
    return mod1(r, 100) 
end

nr (generic function with 1 method)

In [3]:
# Part 1 input is simple here
p1 = 5
p2 = 6
p1_score = 0
p2_score = 0 

d = 1

ws1 = 1000
win_roll = 0
turn = 1

while true
    if turn == 1
        turn = 2
        p1_before = p1
        p1 = mod1(p1+sum([nr(d),nr(d+1),nr(d+2)]), 10)
        p1_score += p1
        win_roll += 3
        #@show (nr(d),nr(d+1),nr(d+2)), p1_before, p1, p1_score
        if p1_score >= ws1
            break
        end
    else
        turn = 1
        p2_before = p2
        p2 = mod1(p2+sum([nr(d),nr(d+1),nr(d+2)]), 10)
        p2_score += p2
        win_roll += 3
        #@show (nr(d),nr(d+1),nr(d+2)), p2_before, p2, p2_score
        if p2_score >= ws1
            break
        end
    end
    d = nr(d+3)
end
@show win_roll, p1_score, p2_score
min(p1_score, p2_score) * win_roll

(win_roll, p1_score, p2_score) = (1098, 913, 1000)


1002474

In [2]:
const winning_score = 22

22

In [5]:
# Part 2
# Min score is 1 each turn, so at most, player 1 will win after 21 turns
# Each turn each player rolls all numbers from 3 to 9
# Can start on any wheel position, can create a map for score count increases each roll
cache = Dict{typeof((1, 2, Int128(0), Int128(0))), typeof(Int128[0, 0])}()
function roll()
    return [d1 + d2 + d3 for d1 in 1:3, d2 in 1:3, d3 in 1:3]
end

function u(p1, p2, s1, s2)
    k = (p1, p2, s1, s2)
    
    if haskey(cache, k)
        return cache[k]
    end
    win_count = Int128[0, 0]
    
    for r in roll()
        p11 = mod1(r + p1, 10)
        s11 = p11 + s1
        
        if s11 >= 21
            win_count[1] += 1
        else
            y, x = u(p2, p11, s2, s11)
            
            win_count[1] += x
            win_count[2] += y
        end
    end
    cache[k] = win_count
    return win_count
end
function roll_and_score(position)
    scores = zeros(Int64, 1,10)
    for d1 in 1:3, d2 in 1:3, d3 in 1:3
        roll = d1+d2+d3
        new_position = mod1(position + d1 + d2 + d3, 10)
        scores[new_position] +=1
    end
    return scores
end
function initialize_state()
    new_state = []
    for i in 1:winning_score + 113
        push!(new_state,zeros(Int64,1,10))
    end
    return new_state
end
function apply_spread(state, score, spread)
    new_state = copy(state)
    for (i, count) in enumerate(spread)
        new_state[score + i][i] += count
    end
    return new_state
end
function roll(state)
    new_state = initialize_state()
    
    for (score, pos_hist) in enumerate(state)
        spread = zeros(Int64,1, 10)
        for (pos, pos_count) in enumerate(pos_hist)
            cur_contribution = roll_and_score(pos) .* pos_count
            spread = spread + cur_contribution
        end
        
        new_state = apply_spread(new_state, score, spread)
    end
    
    return new_state[1:winning_score], sum(sum(new_state[winning_score+1:end]))
end

roll (generic function with 2 methods)

In [7]:
u(5,6,0,0)

2-element Vector{Int128}:
 919758187195363
 635572886949720

In [25]:
p1 = initialize_state()[1:22]
p1[1][4] = 1
p2 = initialize_state()[1:22]
p2[1][8] = 1
    
p1_score = 0
p2_score = 0

while sum(sum(p1)) > 0 || sum(sum(p2)) > 0
    p1, wins = roll(p1)
    p1_score += wins * sum(sum(p2))
    p2, wins = roll(p2)
    p2_score += wins * sum(sum(p1))
end
 p1, wins = roll(p1)
    p1_score += wins * sum(sum(p2))
    p2, wins = roll(p2)
    p2_score += wins * sum(sum(p1))
 p1, wins = roll(p1)
    p1_score += wins * sum(sum(p2))
    p2, wins = roll(p2)
    p2_score += wins * sum(sum(p1))
 p1, wins = roll(p1)
    p1_score += wins * sum(sum(p2))
    p2, wins = roll(p2)
    p2_score += wins * sum(sum(p1))

1596221542559729

In [26]:
@show p1_score
@show p2_score;


p1_score = 2055555562945528
p2_score = 1596221542559729


In [13]:
425958365*425958365

181440528713473225

In [14]:
387287187*387287187

149991365214372969

In [28]:
roll()

3×3×3 Array{Int64, 3}:
[:, :, 1] =
 3  4  5
 4  5  6
 5  6  7

[:, :, 2] =
 4  5  6
 5  6  7
 6  7  8

[:, :, 3] =
 5  6  7
 6  7  8
 7  8  9